# Benchmark FRESA.CAD BSWIMS final Script


This algorithm implementation uses R code and a Python library (rpy2) to connect with it, in order  to run the following it is necesary to have installed both on your computer:

- R (you can download in https://www.r-project.org/) <br>
- install rpy2 by <code> pip install rpy2 </code>

In [1]:
import numpy as np
import pandas as pd
import sys
from pathlib import Path
sys.path.append("../tadpole-algorithms")
import tadpole_algorithms
from tadpole_algorithms.models import Benchmark_FRESACAD_R
from tadpole_algorithms.preprocessing.split import split_test_train_tadpole
#rpy2 libs and funcs
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
from rpy2.robjects import r, pandas2ri 
from rpy2 import robjects

In [2]:
# Load D1_D2 train and possible test data set
data_path_train_test = Path("data/TADPOLE_D1_D2.csv")
data_df_train_test = pd.read_csv(data_path_train_test)

# Load D4 evaluation data set 
data_path_eval = Path("data/TADPOLE_D4_corr.csv")
data_df_eval = pd.read_csv(data_path_eval)

# Split data in test, train and evaluation data
train_df, test_df, eval_df = split_test_train_tadpole(data_df_train_test, data_df_eval)

#instanciate the model to get the functions
model = Benchmark_FRESACAD_R()
#set the flag to true to use a preprocessed data
USE_PREPROC = False


c:\program files\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,72

In [3]:
#preprocess the data
AdjustedTrainFrame,testingFrame,Train_Imputed,Test_Imputed = model.preproc_tadpole_D1_D2(train_df,USE_PREPROC)

[1] "Cortical Thickness Average of RightBankssts"               
[2] "Cortical Thickness Average of RightCaudalAnteriorCingulate"
[3] "Cortical Thickness Average of RightCaudalMiddleFrontal"    
[4] "Cortical Thickness Average of RightBankssts"               
[5] "Cortical Thickness Average of RightCaudalAnteriorCingulate"
[1] "Cortical Thickness Average of LeftBankssts"               
[2] "Cortical Thickness Average of LeftCaudalAnteriorCingulate"
[3] "Cortical Thickness Average of LeftCaudalMiddleFrontal"    
[4] "Cortical Thickness Average of LeftBankssts"               
[5] "Cortical Thickness Average of LeftCaudalAnteriorCingulate"
[1] 1907  782
[1] 371
  [1] "RID"                                             
  [2] "D1"                                              
  [3] "D2"                                              
  [4] "SITE"                                            
  [5] "COLPROT"                                         
  [6] "ORIGPROT"                                

In [4]:
#train and predit 
Forecast_D2 = model.Forecast_D2(AdjustedTrainFrame,testingFrame,Train_Imputed,Test_Imputed,USE_PREPROC)

[1] "C:/Users/jtame/Documents/GitHub/jupyter"
2020-12-21 11:14:05,359 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
2020-12-21 11:14:05,361 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: -
2020-12-21 11:14:05,474 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
2020-12-21 11:14:05,475 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: \
                                                                               - R[write to console]: 
2020-12-21 11:14:05,476 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
 [1]   0   3   6  12  18  24  36  48  60  72  84  96 108 120

       Dementia Dementia to MCI             MCI MCI to Dementia       MCI to NL 
           1356               9            3199             279              61 
             NL  NL to Dementia       NL to MCI 
           2132               1              66 
[1] 908
[1] 

In [5]:
from tadpole_algorithms.evaluation import evaluate_forecast
from tadpole_algorithms.evaluation import print_metrics
# Evaluate the model 
dictionary = evaluate_forecast(eval_df, Forecast_D2)
# Print metrics
print_metrics(dictionary)

[[76 10  0]
 [20 69  3]
 [ 2 11 19]]
mAUC (multiclass Area Under Curve): 0.890
bca (balanced classification accuracy): 0.809
adasMAE (ADAS13 Mean Absolute Error): 5.388
ventsMAE (Ventricles Mean Absolute Error), in % ICV: 0.453
adasWES (ADAS13 Weighted Error Score): 5.490
ventsWES (Ventricles Weighted Error Score ), in % ICV: 0.414
adasCPA (ADAS13 Coverage Probability Accuracy for 50% Confidence Interval: 0.326
ventsCPA (Ventricles Coverage Probability Accuracy for 50% Confidence Interval: 0.280


c:\program files\python38\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=[0, 1, 2] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [6]:
from tadpole_algorithms.preprocessing.split import split_test_train_d3


# Load D3 possible test set
data_path_test = Path("data/TADPOLE_D3.csv")
data_D3 = pd.read_csv(data_path_test)

train_d3f, test_d3f, eval_d3f = split_test_train_d3(train_df,data_D3, data_df_eval)


In [7]:
AdjustedTrainFrame,testingFrame,Train_Imputed,Test_Imputed = model.preproc_tadpole_D3(test_d3f,USE_PREPROC)

I'm here
[1] "I'm here"
[1] "Cortical Thickness Average of RightBankssts"               
[2] "Cortical Thickness Average of RightCaudalAnteriorCingulate"
[3] "Cortical Thickness Average of RightCaudalMiddleFrontal"    
[4] "Cortical Thickness Average of RightBankssts"               
[5] "Cortical Thickness Average of RightCaudalAnteriorCingulate"
[1] "Cortical Thickness Average of LeftBankssts"               
[2] "Cortical Thickness Average of LeftCaudalAnteriorCingulate"
[3] "Cortical Thickness Average of LeftCaudalMiddleFrontal"    
[4] "Cortical Thickness Average of LeftBankssts"               
[5] "Cortical Thickness Average of LeftCaudalAnteriorCingulate"
[1] 383 782
  [1] "D1"                                            
  [2] "D2"                                            
  [3] "SITE"                                          
  [4] "ORIGPROT"                                      
  [5] "EXAMDATE_bl"                                   
  [6] "Years_bl"                            

In [8]:
#train and predit 
Forecast_D3 = model.Forecast_D3(AdjustedTrainFrame,testingFrame,Train_Imputed,Test_Imputed,USE_PREPROC)

2020-12-21 14:02:03,098 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
2020-12-21 14:02:03,116 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: -
2020-12-21 14:02:03,236 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
2020-12-21 14:02:03,237 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: \
                                                                               - R[write to console]: 
2020-12-21 14:02:03,239 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
 [1]  0  3  6 12 18 24 36 48 60 72 84 96

       Dementia Dementia to MCI             MCI MCI to Dementia       MCI to NL 
            476               1             516              62               6 
             NL  NL to Dementia       NL to MCI 
            289               1               8 
[1] 172
[1] 538
[1] 155
[1] 65

  0   1 
 25 187 

 bl m06 m12 m18 m24 m36 m48

In [9]:
from tadpole_algorithms.evaluation import evaluate_forecast
from tadpole_algorithms.evaluation import print_metrics
# Evaluate the model 
dictionary = evaluate_forecast(eval_df,Forecast_D3)

# Print metrics
print_metrics(dictionary)

[[73 13  0]
 [20 63  9]
 [ 2 13 17]]
mAUC (multiclass Area Under Curve): 0.858
bca (balanced classification accuracy): 0.769
adasMAE (ADAS13 Mean Absolute Error): 5.596
ventsMAE (Ventricles Mean Absolute Error), in % ICV: 1.117
adasWES (ADAS13 Weighted Error Score): 5.725
ventsWES (Ventricles Weighted Error Score ), in % ICV: 1.099
adasCPA (ADAS13 Coverage Probability Accuracy for 50% Confidence Interval: 0.126
ventsCPA (Ventricles Coverage Probability Accuracy for 50% Confidence Interval: 0.407


c:\program files\python38\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=[0, 1, 2] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
